# Triangle Ingest

This notebook demonstrates how to ingest triangle data from various external formats into Bermuda. We'll work with Excel and CSV files in different formats: long, wide, and array.

## Setup

In [ ]:
import bermuda as tri
import pandas as pd
import altair as alt
from datetime import date
import os

# Enable HTML rendering
alt.renderers.enable("html")

# Check that our data files exist
if not os.path.exists('data/excel/triangle_data.xlsx'):
    print("Data files not found. Running data generation script...")
    !python create_excel_data.py
else:
    print("Data files found.")

## Understanding Data Formats

Bermuda supports three main tabular formats for triangle data:

1. **Long Format**: Each row represents a single cell value
2. **Wide Format**: Each row represents all values for a cell
3. **Array Format**: Traditional actuarial triangle layout

## 1. Long Format Ingestion

Long format is the most flexible - each row contains one value for one cell.

In [ ]:
# Load and examine the long format CSV
gl_long_df = pd.read_csv('data/excel/gl_long.csv')
print("Long format structure (first 5 rows):")
display(gl_long_df.head())
print(f"\nShape: {gl_long_df.shape}")
print(f"Columns: {list(gl_long_df.columns)}")

In [ ]:
# Ingest long format CSV
gl_triangle = tri.long_csv_to_triangle('data/excel/gl_long.csv')
print("Triangle loaded from long CSV:")
display(gl_triangle)

In [ ]:
# Note the renamed columns - we need to fix the mapping
# The CSV has 'paid_losses' but Bermuda expects 'paid_loss'
# Let's reload with proper column mapping

# First, load as DataFrame and rename columns
gl_df = pd.read_csv('data/excel/gl_long.csv')
gl_df = gl_df.rename(columns={
    'paid_losses': 'paid_loss',
    'incurred_losses': 'reported_loss',
    'earned_prem': 'earned_premium'
})

gl_df['period_start'] = pd.to_datetime(gl_df['period_start'])
gl_df['period_end'] = pd.to_datetime(gl_df['period_end'])
gl_df['evaluation_date'] = pd.to_datetime(gl_df['evaluation_date'])

# Now ingest from DataFrame
gl_triangle = tri.long_data_frame_to_triangle(gl_df)
print("Triangle with corrected column names:")
print(gl_triangle)

## 2. Wide Format Ingestion

Wide format has one row per cell with all fields as columns.

In [ ]:
# Load wide format from Excel
ca_wide_df = pd.read_excel('data/excel/triangle_data.xlsx', sheet_name='ca_wide_format')
print("Wide format structure (first 5 rows):")
display(ca_wide_df.head())
print(f"\nShape: {ca_wide_df.shape}")

In [ ]:
# Ingest wide format
ca_triangle = tri.wide_data_frame_to_triangle(ca_wide_df, field_cols=['paid_loss','reported_loss','earned_premium'])
print("Triangle loaded from wide format:")
display(ca_triangle)

## 3. Array Format Ingestion

Array format is the traditional actuarial triangle layout with accident periods as rows and development periods as columns.

Here we show some messier data, as it actually exists in the real world

In [ ]:
# Load the messy array format from Excel - all three triangles in one sheet!
# This mimics real-world data where multiple triangles are stacked
import pandas as pd

# Read the entire sheet
pa_raw_df = pd.read_excel('data/excel/triangle_data.xlsx', sheet_name='pa_array_format', header=None)
print("Raw messy sheet structure (first 15 rows):")
display(pa_raw_df.head(18))
print(f"\nTotal sheet shape: {pa_raw_df.shape}")

# We need to parse this messy format and extract the three triangles
print("\nLet's see what we're dealing with - finding the section headers...")

In [ ]:
# Parse the messy Excel sheet to extract the three triangles
def parse_messy_triangles(df):
    """Parse a messy Excel sheet with multiple triangles separated by blank rows"""
    # Find the section headers
    sections = {}
    current_section = None
    section_start = None
    
    for idx, row in df.iterrows():
        # Check if this row contains a section header
        if pd.notna(row.iloc[0]) and isinstance(row.iloc[0], str):
            header_text = str(row.iloc[0]).upper().strip()
            if any(keyword in header_text for keyword in ['PAID LOSS', 'REPORTED LOSS', 'EARNED PREMIUM']):
                # End previous section if it exists
                if current_section is not None and section_start is not None:
                    sections[current_section] = (section_start, idx - 1)
                
                # Start new section
                if 'PAID' in header_text:
                    current_section = 'paid_loss'
                elif 'REPORTED' in header_text:
                    current_section = 'reported_loss'
                elif 'EARNED' in header_text or 'PREMIUM' in header_text:
                    current_section = 'earned_premium'
                section_start = idx + 1
    
    # Don't forget the last section
    if current_section is not None and section_start is not None:
        sections[current_section] = (section_start, len(df) - 1)
    
    print(f"Found sections: {list(sections.keys())}")
    
    # Extract each triangle
    triangles = {}
    for field, (start_row, end_row) in sections.items():
        print(f"\nExtracting {field} from rows {start_row} to {end_row}")
        
        # Get the data for this section
        section_df = df.iloc[start_row:end_row + 1].copy()
        
        # Remove completely empty rows
        section_df = section_df.dropna(how='all')
        
        # The first row should be the header
        if len(section_df) > 0:
            # Use first row as header
            section_df.columns = section_df.iloc[0]
            section_df = section_df.drop(section_df.index[0])
            
            # Reset index and clean up
            section_df = section_df.reset_index(drop=True)
            section_df.columns.name = None
            
            # Rename first column to 'period' if it looks like accident_period  
            if section_df.columns[0] == 'accident_period' or pd.isna(section_df.columns[0]):
                new_cols = ['period'] + [col for col in section_df.columns[1:]]
                section_df.columns = new_cols
            
            triangles[field] = section_df
            print(f"  Shape: {section_df.shape}")
            print(f"  Columns: {list(section_df.columns)}")
    
    return triangles

# Parse the messy sheet
triangle_dfs = parse_messy_triangles(pa_raw_df)

# Show what we extracted
for field, df in triangle_dfs.items():
    print(f"\n{field.upper()} triangle:")
    display(df.head(3))

In [ ]:
# Now convert each parsed triangle to Bermuda format and merge them
triangles = []

for field, df in triangle_dfs.items():
    print(f"\nConverting {field} triangle...")
    
    # Clean up the DataFrame structure before conversion
    clean_df = df.copy()
    
    # The DataFrame has duplicate 'period' columns - we need the second one with dates
    # Remove the header row that says 'accident_period'
    clean_df = clean_df[clean_df.iloc[:, 0] != 'accident_period']
    
    # Remove any rows that are completely empty
    clean_df = clean_df.dropna(how='all')
    
    # Reset index
    clean_df = clean_df.reset_index(drop=True)
    
    # The second column (index 1) contains the actual dates, use that as 'period'
    if len(clean_df.columns) > 1:
        # Create new DataFrame with proper structure
        date_col = clean_df.iloc[:, 1]  # Second column has the dates
        data_cols = clean_df.iloc[:, 2:]  # Rest are data columns
        
        # Build new DataFrame
        new_df = pd.DataFrame()
        new_df['period'] = date_col
        
        # Add data columns with proper names
        for i, col in enumerate(data_cols.columns):
            new_df[f"dev_{col}"] = data_cols.iloc[:, i]
        
        # Remove any rows where period is NaN
        new_df = new_df.dropna(subset=['period'])
        
        print(f"  Cleaned DataFrame shape: {new_df.shape}")
        print(f"  First few periods: {new_df['period'].head(3).tolist()}")
        
        # Convert to Bermuda triangle
        triangle = tri.array_data_frame_to_triangle(
            new_df,
            field=field,
            period_resolution=12,  # Annual periods
            eval_resolution=12     # Annual evaluations
        )
        triangles.append(triangle)
        print(f"  Created triangle with {len(triangle)} cells")

# Merge all triangles into one with multiple fields
if triangles:
    pa_triangle = triangles[0]
    for t in triangles[1:]:
        pa_triangle = tri.merge(pa_triangle, t)
    
    print(f"\n🎉 Successfully created PA triangle from messy Excel data!")
    print(f"Triangle has {len(pa_triangle)} cells")
    print(f"Available fields: {pa_triangle.fields}")
    display(pa_triangle)
else:
    print("❌ Failed to extract triangles from the messy data")

In [ ]:
# Demonstrate the power of having multiple fields in one triangle
# Calculate loss ratios and case reserves automatically
print("Sample cells showing all fields:")
for i, cell in enumerate(pa_triangle[:3]):
    print(f"\nCell {i+1}:")
    print(f"  Period: {cell.period_start} to {cell.period_end}")
    print(f"  Evaluation: {cell.evaluation_date}")
    print(f"  Paid Loss: ${cell.values.get('paid_loss', 0):,.0f}")
    print(f"  Reported Loss: ${cell.values.get('reported_loss', 0):,.0f}")
    print(f"  Earned Premium: ${cell.values.get('earned_premium', 0):,.0f}")
    
    # Calculate derived metrics
    if cell.values.get('earned_premium', 0) > 0:
        paid_lr = cell.values.get('paid_loss', 0) / cell.values.get('earned_premium', 1)
        reported_lr = cell.values.get('reported_loss', 0) / cell.values.get('earned_premium', 1)
        print(f"  Paid Loss Ratio: {paid_lr:.1%}")
        print(f"  Reported Loss Ratio: {reported_lr:.1%}")
    
    if cell.values.get('reported_loss', 0) > cell.values.get('paid_loss', 0):
        case_reserves = cell.values.get('reported_loss', 0) - cell.values.get('paid_loss', 0)
        print(f"  Case Reserves: ${case_reserves:,.0f}")

## Interactive Exercise: Load Your Own Data

Now it's your turn! Complete the code below to load triangle data in different formats.

### Exercise 1: Load Wide Format CSV

Complete the code to load the CA wide format from CSV:

In [ ]:
# TODO: Load ca_wide.csv using the appropriate Bermuda function
# Hint: Use tri.wide_csv_to_triangle()

# ca_triangle_csv = tri.________('data/excel/ca_wide.csv')
# display(ca_triangle_csv)

### Exercise 2: Load Array Format from CSV

Complete the code to load PA array data from CSV:

In [ ]:
# TODO: Load pa_array.csv and convert to triangle
# Remember: array format needs the field name!

# pa_csv_df = pd.read_csv('data/excel/pa_array.csv')
# pa_triangle_csv = tri.array_data_frame_to_triangle(
#     pa_csv_df,
#     field='______',  # What field should go here?
#     period_resolution=___,  # Annual = ?
#     eval_resolution=___     # Annual = ?
# )
# display(pa_triangle_csv)

### Exercise 3: Handle Multiple Fields in Array Format

Load both paid and reported losses from array format:

In [ ]:
# TODO: Create a triangle with multiple fields from array format
# Hint: Use tri.array_triangle_builder() with lists of DataFrames and field names

# Load a sample triangle to get both fields
# sample = tri.binary_to_triangle('data/excel/ca_filtered.trib')
# 
# # Export to array format for both fields
# paid_array = tri.triangle_to_array_data_frame(sample, field='paid_loss')
# reported_array = tri.triangle_to_array_data_frame(sample, field='reported_loss')
# 
# # Now build a multi-field triangle
# multi_triangle = tri.array_triangle_builder(
#     dfs=[_____, _____],  # List of DataFrames
#     fields=['_____', '_____'],  # Corresponding field names
#     period_resolution=12,
#     eval_resolution=12
# )
# display(multi_triangle)

## Creating Multi-Slice Triangles

Now let's combine our three triangles (GL, CA, PA) into a single multi-slice triangle.

In [ ]:
# Add metadata to distinguish the slices
gl_cells = []
for cell in gl_triangle:
   # Create new metadata with same standard fields
   new_meta = tri.Metadata(
       risk_basis=cell.metadata.risk_basis,
       reinsurance_basis=cell.metadata.reinsurance_basis,
       loss_definition=cell.metadata.loss_definition,
   )
   # Copy existing details and add new field
   new_meta.details.update(cell.metadata.details)
   new_meta.details['line'] = 'GL'
   gl_cells.append(cell.replace(metadata=new_meta))
gl_with_meta = tri.Triangle(gl_cells)

ca_cells = []
for cell in ca_triangle:
   # Create new metadata with same standard fields
   new_meta = tri.Metadata(
       risk_basis=cell.metadata.risk_basis,
       reinsurance_basis=cell.metadata.reinsurance_basis,
       loss_definition=cell.metadata.loss_definition,
   )
   # Copy existing details and add new field
   new_meta.details.update(cell.metadata.details)
   new_meta.details['line'] = 'CA'
   ca_cells.append(cell.replace(metadata=new_meta))
ca_with_meta = tri.Triangle(ca_cells)

pa_cells = []
for cell in pa_triangle:
   # Create new metadata with same standard fields
   new_meta = tri.Metadata(
       risk_basis=cell.metadata.risk_basis,
       reinsurance_basis=cell.metadata.reinsurance_basis,
       loss_definition=cell.metadata.loss_definition,
   )
   # Copy existing details and add new field
   new_meta.details.update(cell.metadata.details)
   new_meta.details['line'] = 'PA'
   pa_cells.append(cell.replace(metadata=new_meta))
pa_with_meta = tri.Triangle(pa_cells)

# Combine into multi-slice triangle
combined = gl_with_meta + ca_with_meta + pa_with_meta
print("Combined multi-slice triangle:")
display(combined)
print(f"\nNumber of slices: {len(combined.slices)}")


In [ ]:
combined.plot_right_edge()


## Saving Triangle Data

Bermuda supports multiple output formats for saving your work.

In [ ]:
# Save to CSV (long format)
tri.triangle_to_long_csv(combined, 'data/excel/combined_long.csv')
print("Saved to long CSV: data/excel/combined_long.csv")

# Save to CSV (wide format)
tri.triangle_to_wide_csv(combined, 'data/excel/combined_wide.csv')
print("Saved to wide CSV: data/excel/combined_wide.csv")

In [ ]:
# Save to JSON
tri.triangle_to_json(combined, 'data/excel/combined.json')
print("Saved to JSON: data/excel/combined.json")

# Peek at the JSON structure
import json
with open('data/excel/combined.json', 'r') as f:
    json_data = json.load(f)
    print(json.dumps(json_data, indent=2, default=str)[:500] + "...")

In [ ]:
# Save to binary trib format (most efficient)
tri.triangle_to_binary(combined, 'data/excel/combined.trib')
print("Saved to binary trib: data/excel/combined.trib")

# Compare file sizes
import os
sizes = {
    'CSV (long)': os.path.getsize('data/excel/combined_long.csv'),
    'CSV (wide)': os.path.getsize('data/excel/combined_wide.csv'),
    'JSON': os.path.getsize('data/excel/combined.json'),
    'Trib (binary)': os.path.getsize('data/excel/combined.trib')
}

print("\nFile size comparison:")
for format_name, size in sizes.items():
    print(f"  {format_name}: {size:,} bytes")

print(f"\nBinary format is {sizes['JSON'] / sizes['Trib (binary)']:.1f}x smaller than JSON")

## Summary

In this notebook, we've covered:

1. **Data Format Types**: Long, wide, and array formats each have their use cases
2. **Ingestion Methods**: Different functions for each format
3. **Column Mapping**: Aligning external column names with Bermuda conventions
4. **Multi-Slice Triangles**: Combining triangles from different sources
5. **Export Options**: CSV, JSON, and binary formats with different trade-offs

The binary trib format is Ledger's proprietary format that:
- Saves space (typically 5-10x smaller than JSON)
- Loads faster (no parsing overhead)
- Preserves all metadata and structure perfectly
- Works seamlessly across Bermuda versions

### Answer Key for Exercises

**Exercise 1:**
```python
ca_triangle_csv = tri.wide_csv_to_triangle('data/excel/ca_wide.csv')
```

**Exercise 2:**
```python
pa_csv_df = pd.read_csv('data/excel/pa_array.csv')
pa_triangle_csv = tri.array_data_frame_to_triangle(
    pa_csv_df,
    field='paid_loss',
    period_resolution=12,
    eval_resolution=12
)
```

**Exercise 3:**
```python
multi_triangle = tri.array_triangle_builder(
    dfs=[paid_array, reported_array],
    fields=['paid_loss', 'reported_loss'],
    period_resolution=12,
    eval_resolution=12
)
```